In [5]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

     ---------------------------------------- 5.8/5.8 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 182.4/182.4 kB 1.6 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [11]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [20]:
tokens = tokenizer.encode('It was bad but very bad. Great', return_tensors='pt')

In [21]:
result = model(tokens)

In [22]:
result.logits

tensor([[ 1.2880,  1.1319,  0.5780, -0.9284, -1.7535]],
       grad_fn=<AddmmBackward0>)

In [23]:
int(torch.argmax(result.logits))+1

1

In [24]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [28]:
print(reviews)

['Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly', "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.", 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well', "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!", 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.', "It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast

In [29]:
import numpy as np
import pandas as pd

In [31]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [34]:
df['review'].iloc[4]

'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.'

In [35]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [37]:
sentiment_score(df['review'].iloc[4])

5

In [38]:

df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [42]:
df.head(11)

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5
5,It was ok. The coffee wasn't the best but it w...,3
6,We came for brunch twice in our week-long visi...,4
7,Ron & Jo are on the go down under and Wow! We...,5
8,I went here a little while ago- a beautiful mo...,2
9,Great coffee and vibe. That's all you need. C...,5


In [41]:
df.shape

(11, 2)